# Create SageMaker session and role

Here my make use of the execution role we gave the sagemaker notebook instance. This role provides access to the S3 bucket we created. 

In [ ]:
# S3 prefix
s3_bucket = 'srt-sm'

#prefix = 'Scikit-LinearLearner-pipeline-srt' #use this for a smaller subset of the training data
prefix = 'Sklearn-RandomizedGridSearch' #this is all 993 labeled samples

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

# Create a SageMaker Scikit estimator 
Here we create an instance of a `sagemaker.sklearn.estimator.sklearn` estimator. The constructor accepts several constructor arguments:

 - **source_dir**: Path (absolute or relative) to the directory with our custom model training source code.
 - **entry_point**: The path to the Python script SageMaker runs for training and prediction within `source_dir`.
 - **role**: Role ARN, which is provided by `get_execution_role()`
 - **train_instance_type (optional)**: The type of SageMaker instances for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types. Also, note that you may need to request an EC2 quota increase for these ml ec2 instance types.
 - **sagemaker_session (optional)**: The session used to train on Sagemaker, as returned by `sagemaker.Session()`.
 - **output_path (optional)**: s3 location where you want the training result (model artifacts and optional output files) saved. If not specified, results are stored to a default bucket. If the bucket with the specific name does not exist, the estimator creates the bucket during execution of the `fit()` method.
 - **train_max_run (optional)**: Timeout in seconds for training (default: 24 * 60 * 60). After this amount of time Amazon SageMaker terminates the job regardless of its current status.


In [ ]:
from sagemaker.sklearn.estimator import SKLearn

entry_point = 'sklearn_featureizer.py'
source_dir = 'pipeline'

s3_output_key_prefix = "training_output"
model_output_path = 's3://{}/{}/{}/{}'.format(s3_bucket, prefix, s3_output_key_prefix, 'model')

# terminate model training after 48 hours
train_max_run = 48 * 60 * 60

grid_search = SKLearn(source_dir=source_dir,
                      entry_point=entry_point,
                      role=role,
                      train_instance_type="ml.c5.4xlarge",
                      sagemaker_session=sagemaker_session,
                      output_path=model_output_path,
                      train_max_run=train_max_run)

train_input = f's3://{s3_bucket}/{prefix}/srt_train.csv'

In [ ]:
# Note that this will take awhile.
grid_search.fit({'train': train_input}, logs=True)

# Deploy the Model

We now have a fitted model (i.e. the best estimator from the Grid Search) in our s3 bucket. We can now deploy this model behind a single endpoint. When this is done, you'll be able to see this endpoint under **Endpoints** in the SageMaker console.

In [ ]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_name = 'inference-pipeline-' + timestamp_prefix
endpoint_name = 'inference-pipeline-ep-' + timestamp_prefix

model = grid_search.create_model(role=role)

model.deploy(initial_instance_count=1, 
             instance_type='ml.c5.xlarge',
             endpoint_name=endpoint_name)

# Request inferences from the endpoint 
With our model deployed behind a REST API, we'll now make some requests to it in order to get inferences from our validation set. We can then use these inferences to see how well the trained model performs on out-of-sample data.

Note that we need to make our request with the payload in `text/csv` format, since that is what our script currently supports (see `input_fn()` in our entrypoint file). If other formats need to be supported, this would have to be added to that `input_fn()` function. Note, however, that we set the `accept` to `application/json` to get our output, i.e. the inferences, that way. We do this because our `ouput_fn()` function returns JSON.

In [ ]:
import pandas as pd
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

#read in the validation set from the s3 bucket
test_input = f's3://{s3_bucket}/{prefix}/srt_test.csv'

#convert it to a pandas dataframe
test_df = pd.read_csv(test_input)

test_df.columns = ['target', 'text']

def format_as_csv(text):
    #since we make our request with the payload in text/csv format, we need to sanitize the text first
    return text.replace(",","").replace("\n","")

test_df['text'] = test_df['text'].apply(format_as_csv)

In [ ]:
import time

predictor = RealTimePredictor(endpoint=endpoint_name,
                          sagemaker_session=sagemaker_session,
                          serializer=csv_serializer,
                          content_type=CONTENT_TYPE_CSV,
                          accept=CONTENT_TYPE_JSON)


predictions = []
samples_to_drop = []
for i,v in test_df['text'].iteritems():
    try:
        pred = predictor.predict(v)
        predictions.append(pred)
    except Exception as e:
        # RealTimePredictor will timeout after 60 seconds and some docs are so large that this occurs
        # Since timeouts can be managed in other endpoint deployments and usage patterns,
        # we'll pass over this one error for now
        print(f"response for the {i}th sample took too long:\n{e}")
        samples_to_drop.append(i)
        #sleep while server reboots
        time.sleep(10)
        pass


In [ ]:
import json

y_pred = []
for i, p in enumerate(predictions):
    result = json.loads(p)
    pred = result.get("instances")[0].get('prediction')
    y_pred.append(pred)

y_true = test_df['target'].tolist()
y_true.pop(samples_to_drop[0])

# Evaluate model performance on the validation set

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true, y_pred)

In [ ]:
from sklearn.metrics import classification_report

clf_report = classification_report(y_true, y_pred)
print(clf_report)

# Delete Endpoint 
Once we are finished with the endpoint, we clean up the resources since the endpoint incurs costs for as long as it is alive.

In [ ]:
sm_client = sagemaker_session.boto_session.client('sagemaker')
sm_client.delete_endpoint(EndpointName=endpoint_name)